In [1]:
# install necessary libraries
! pip install sqlalchemy # ==1.4.46
! pip install PyMySQL

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass() # insert your password below after executing this cell

········


In [3]:
# make the connection with the database
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila' # database_location/database_name
engine = create_engine(connection_string)

In [ ]:
data = pd.read_sql_query('SELECT * FROM loan', engine)
data.head()

In [13]:
# Create a Python connection with SQL database and retrieve the results of the last two queries 

query1 = '''
SELECT c.customer_id, CONCAT(c.first_name, ' ', c.last_name) AS customer_name, COUNT(*) AS rental_count
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
WHERE MONTH(r.rental_date) = 5
GROUP BY c.customer_id, customer_name
'''

query2 = '''
SELECT c.customer_id, CONCAT(c.first_name, ' ', c.last_name) AS customer_name, COUNT(*) AS rental_count
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
WHERE MONTH(r.rental_date) = 6
GROUP BY c.customer_id, customer_name
'''

In [14]:
df_may = pd.read_sql_query(query1, engine)
df_june = pd.read_sql_query(query2, engine)

In [8]:
# Check the number of rentals for each customer for May 

print("Result of Query 1 (May):")
print(df_may)

Result of Query 1 (May):
     customer_id       customer_name  rental_count
0              1          MARY SMITH             2
1              2    PATRICIA JOHNSON             1
2              3      LINDA WILLIAMS             2
3              5     ELIZABETH BROWN             3
4              6      JENNIFER DAVIS             3
..           ...                 ...           ...
515          594       EDUARDO HIATT             4
516          595  TERRENCE GUNDERSON             1
517          596    ENRIQUE FORSYTHE             6
518          597      FREDDIE DUGGAN             2
519          599      AUSTIN CINTRON             1

[520 rows x 3 columns]


In [9]:
# Check the number of rentals for each customer for June

print("Result of Query 2 (June):")
print(df_june)

Result of Query 2 (June):
     customer_id       customer_name  rental_count
0              1          MARY SMITH             7
1              2    PATRICIA JOHNSON             1
2              3      LINDA WILLIAMS             4
3              4       BARBARA JONES             6
4              5     ELIZABETH BROWN             5
..           ...                 ...           ...
585          595  TERRENCE GUNDERSON             2
586          596    ENRIQUE FORSYTHE             2
587          597      FREDDIE DUGGAN             3
588          598       WADE DELVALLE             1
589          599      AUSTIN CINTRON             4

[590 rows x 3 columns]


In [26]:
# Write function that checks if customer borrowed more or less films in the month of June as compared to May

def compared_rental_count(df_may, df_june, customer_id):

# merge the DFs on customer_id
    merged_df = pd.merge(df_may, df_june, on='customer_id', suffixes=('_may', '_june'))

# filter the merged DF for specific customer
    customer_df = merged_df[merged_df['customer_id'] == customer_id]
    
# get rental counts for may/june
    rental_count_may = customer_df['rental_count_may'].iloc[0]
    rental_count_june = customer_df['rental_count_june'].iloc[0]
    
# compare rental results, if/else
    if rental_count_june > rental_count_may:
        result = f"Customer number {customer_id} borrowed more films in June than May."
    elif rental_count_june < rental_count_may:
        result = f"Customer number {customer_id} borrowed fewer films in June than May."
    else:
        result = f"Customer number {customer_id} borrowed the same number of films in June and May."
    return result

# example of customer with ID number 123:
customer_id = 123 
result = compared_rental_count(df_may, df_june, customer_id)
print(result)

Customer number 123 borrowed more films in June than May.
